       Scheduling and automating code via Airflow can be done with this program. This is a general source-code, since 
       the task never asked to automate the pipeline specifially, I didn't do it. I am also partially new to Airflow
       (started learning 1.5 week before), which is why I am not an expert at the moment to complete the whole task
       with Airflow, in 2 weeks from today I will be much better with dags. I have still written this for keeping in mind 
       the structure of the pipeline in my main code (75-61-25)
                    

In [11]:
#importing libraries
import time
from datetime import datetime
#from airflow.models.dag import DAG
#from airflow.decorators import task
#from airflow.utils.task_group import TaskGroup
#from airflow.providers.microsoft.mssql.hooks.mssql import MsSqlHook
#from airflow.hooks.base_hook import BaseHook
#import pandas as pd
#from sqlalchemy import create_engine

In [13]:
#extract()
#task()
def extract():
    #connection defined in airflow
    hook= MsSqlHook(mssql_conn_id="sqlserver")
    #sql to get tables like previous script
    sql=""" SELECT t.name as table_name from sys.tables t where t.name in
    ('df_contracts1','df_products1','df_prices1')"""
    #saving in df variable
    df= hook.get_pandas_df(sql)
    #print(df)
    #since we can only share serial data, I have converted it to dictionary
    tbl_dict=df.to_dict('dict')
    return tbl_dict

In [20]:
#task()
def load():
    #Saving connection in conn variable
    conn = Basehook.get_connection('postgres')
    engine = create_engine(f'postgresql://{conn.login}:{conn.password}@{conn.host}:{conn.port}/{conn.schema}')
    all_tbl_name = []
    start_time = time.time()
    #access to tables defined in dictionaries
    for i,j in tbl_dict['table_name'].items():
        #print(v)
        all_tbl_name.append(v)
        rows_imported = 0
        sql = f'select * FROM {v}'
        hook = MsSqlHook(mssql_conn_id = "sqlserver")
        df = hook.get_pandas_df(sql)
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}...for table{v}')
        df.to_sql(f'stg_{v}',engine, if_exists = 'replace', index= False)
        rows_imported += len(df)
        print(f'Done. {str(round(time.time() - start_time,2))} total seconds elapsed')
    print("Data imported successfully:")
    return all_tbl_name
    
    